<a href="https://colab.research.google.com/github/Ruanvans/ECG-classification-using-deep-learning/blob/master/mitbih_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importation of necessary libraries and tools**

In [0]:
%tensorflow_version 1.x

In [0]:
import numpy as np 
import pandas as pd 
import keras 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
from keras.models import Sequential 
from keras.layers import Reshape, Dense, Activation, Flatten, Convolution1D, Dropout, MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D
from keras.utils import to_categorical
from sklearn.utils import resample, shuffle 
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.optimizers import rmsprop
from sklearn.metrics import roc_curve, auc ,precision_score, recall_score, confusion_matrix
from sklearn.metrics import classification_report, roc_auc_score
import itertools
from itertools import cycle 
import random
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM,Bidirectional, GRU
from keras import optimizers,regularizers
from keras.layers.normalization import BatchNormalization
from scipy import interp

# **Helper methods to visualize results**

In [0]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
   
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [0]:
def model_evalutation(history, test, y_test, model):
    score = model.evaluate((test),y_test)
    print("Model Accuracy: %.2f%%" % (score[1]*100))

    plt.figure(figsize=(15,5))

    # Plot training & validation accuracy values
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')

    plt.suptitle("Model Accuracy and Model Loss")
    plt.tight_layout()
    plt.show()

In [0]:
def roc_curve_evaluation(y,y_pred, y_test):

    y = label_binarize(y, classes=[0, 1])
    n_classes = 2

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    plt.figure(figsize=(10,10))
    lw = 2
    plt.plot(fpr["micro"], tpr["micro"],
            label='micro-average ROC curve (area = {0:0.2f})'
                  ''.format(roc_auc["micro"]),
            color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
            label='macro-average ROC curve (area = {0:0.2f})'
                  ''.format(roc_auc["macro"]),
            color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                label='ROC curve of class {0} (area = {1:0.2f})'
                ''.format(i, roc_auc[i]))


    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.show()

# **Data preparation and preprocessing**

In [0]:
#importation of datasets
df_mitbih_train = pd.read_csv('mitbih_train.csv', header=None)
df_mitbih_test = pd.read_csv('mitbih_test.csv', header=None)

#concatenation of the normal and abnormal datasets 
frames = [df_ptb1, df_ptb2]
df_ptb3 = pd.concat(frames) 
#Does a value count of the newly made concatenated dataset
df_ptb3[187] = df_ptb3[187].astype(int)
equilibre = df_ptb3[187].value_counts()
# Resampling Dataset
df_1 = df_ptb3[df_ptb3[187] == 1]
df_0 =(df_ptb3[df_ptb3[187] == 0]).sample(n=4000,random_state=42)

df_1_upsample=resample(df_1,replace=True,n_samples=4000,random_state=123)
train_df=pd.concat([df_0,df_1_upsample])
#shuffle method used here to ensure that models are correctly trained and tested
train_df= shuffle(train_df)
#Value count on the resampled dataset
train_df[187].value_counts()

resampled = train_df[187].value_counts()

# **Basic neural net to establish a baseline accuracy**

In [0]:
model_1 = Sequential()
model_1.add(Dense(100, input_dim=187, activation='relu'))
model_1.add(Dense(100, activation='relu'))
model_1.add(Dense(90, activation='relu' ))
model_1.add(Dense(70, activation='relu'))
model_1.add(Dense(2, activation='sigmoid'))

print(model_1.summary())
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]

history = model_1.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=callbacks ,epochs=100, batch_size=32)

In [0]:
test = X_test
model = model_1
model_evalutation(history, test, y_test, model)

In [0]:
y_pred = model_1.predict(X_test, batch_size=1000)
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cnf_matrix, classes=['1','0'],
                      title='Confusion matrix, without normalization')
plt.show()

In [0]:
y = y
y_pred = model_1.predict(X_test, batch_size=1000)
y_test = y_test

roc_curve_evaluation(y, y_pred, y_test)

# **Original convolutional neural network**

In [0]:
model_2 = Sequential()
model_2.add(Convolution1D(100, 5, activation='relu', input_shape=(187, 1)))
model_2.add(Convolution1D(110, 5, activation='relu'))
model_2.add(Convolution1D(120, 5, activation='relu'))
model_2.add(MaxPooling1D(3))
model_2.add(Convolution1D(140, 10, activation='relu'))
model_2.add(Convolution1D(150, 10, activation='relu'))
model_2.add(Convolution1D(160, 10, activation='relu'))
model_2.add(Convolution1D(170, 10, activation='relu'))
model_2.add(Convolution1D(200, 10, activation='relu'))
model_2.add(GlobalAveragePooling1D())
model_2.add(Dropout(0.5))
model_2.add(Dense(100, activation='relu'))
model_2.add(Dense(50, activation='relu' ))
model_2.add(Dense(2, activation='softmax'))

print(model_2.summary())
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

history = model_2.fit(Xc_train, y_train, validation_data=(Xc_test, y_test), callbacks=callbacks, epochs=40, batch_size= 32)

In [0]:
test = Xc_test
y_test = y_test
model = model_2
model_evalutation(history, test, y_test, model)

In [0]:
y_pred = model_2.predict(Xc_test, batch_size=1000)
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cnf_matrix, classes=['0','1'],
                      title='Confusion matrix, without normalization')
plt.show()


In [0]:
y = y 
y_pred = model_2.predict(Xc_test, batch_size=1000)
y_test = y_test

roc_curve_evaluation(y, y_pred, y_test)

# **Well known convolutional neural network for comparison purposes**

In [0]:
model_3 = Sequential()
model_3.add(Convolution1D(6, 3, activation='relu', input_shape=(187, 1), padding='same'))
model_3.add(AveragePooling1D(2))
model_3.add(Convolution1D(16, 3, activation='relu', padding='valid'))
model_3.add(AveragePooling1D(2))
model_3.add(Flatten())
model_3.add(Dense(120, activation='relu'))
model_3.add(Dense(84, activation='relu'))
model_3.add(Dense(2, activation='softmax'))

print(model_3.summary())
model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

history = model_3.fit(Xc_train, y_train, validation_data=(Xc_test, y_test), callbacks=callbacks , epochs=40, batch_size= 32)

In [0]:
test = Xc_test
model = model_3
model_evalutation(history, test, y_test, model)

In [0]:
y_pred = model_3.predict(Xc_test, batch_size=1000)
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cnf_matrix, classes=['0','1'],
                      title='Confusion matrix, without normalization')
plt.show()

In [0]:
y = y
y_pred = model_3.predict(Xc_test, batch_size=1000)
y_test = y_test

roc_curve_evaluation(y, y_pred, y_test)